In [574]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)
library(ggplot2)
library(dummies)
library(stringr)
library(rms)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_score_finale.Rdata")
df <- df_score_finale
df <- subset(df,select= -c(degenzaPostTraOSP))

In [575]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

In [414]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +53 secs: 

 confirmed 19 attributes: Admsite, Albumin, Ascites, Centre, CRIO and 14 more;

 rejected 29 attributes: Age, Alcoholic, Anticoagulation, Cholestatic, cirrhosisDrugs and 24 more;

 still have 25 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +1.1 mins: 

 confirmed 1 attribute: Encephalopathy;

 rejected 5 attributes: Neurocomorbidities, Periodstx, Previoussurg, Renalc

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "Fibrinogen"       "CRIO"             "Crystalloids"     "PPS5"            
[17] "Norepinend"       "Lactates"         "ReperfusSy"       "Biliaryanasth"   
[21] "Openabdomen"      "DonorSodiemia"    "DonorGGT"         "MELDNa"          
[25] "Nocomorbodities"  "Admsite"          "Encephalopathy"   "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [214]:
par(mar = c(13, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.3, las=2, xlab="", main="Variable Importance")

In [215]:
df_boruta$deg_cat <- df$deg_cat

In [278]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "Fibrinogen"       "CRIO"             "Crystalloids"     "PPS5"            
[17] "Norepinend"       "Lactates"         "ReperfusSy"       "Biliaryanasth"   
[21] "Openabdomen"      "DonorSodiemia"    "DonorGGT"         "MELDNa"          
[25] "Nocomorbodities"  "Admsite"          "Encephalopathy"   "HCC"

In [576]:
df$deg_cat_bin = ifelse(df$deg_cat == 'NotFast',1,0)

df <- subset(df, select = -deg_cat)

In [577]:
df <- df[df$PRBCintraop < 52,] # 4 big outliers

In [578]:
df <- subset(df, select = c('MELDNa','PRBCintraop','Duration','Centre','deg_cat_bin'))

In [579]:
set.seed(1)
df$dataset <- sample(x=c("train","validate"),
size=nrow(df),
replace=TRUE,
prob=c(0.75,0.25))

## LOESS smoothing curves for numeric variables

In [580]:
df.cont<- df[df$dataset=="train",
c('deg_cat_bin','MELDNa','PRBCintraop','Duration')]


df.cat<- df[df$dataset=="train",
c('Centre')]

ymark<-seq(0,1,0.1)

for(var in names(df.cont)[-1]){
xvar<-seq(min(df.cont[,var]),
max(df.cont[,var]),
length.out=10)
    
    
mypath <- file.path("C:/Users/abonini/Documents/score",
paste(paste("plot",var,sep = "_"),
"pdf", sep = "."))

pdf(file=mypath)

gg<-ggplot(df.cont, aes(x=df.cont[,var],y=deg_cat_bin))+
geom_jitter(size=1, alpha=0.2,height=0.05)+
stat_smooth(method="loess",colour="blue", size=1.5)+
xlab(var)+
ylab("Probability on length of hospital stay")+
theme_bw()+
geom_hline(yintercept=ymark,col="red")+
scale_y_continuous(breaks=ymark)+
geom_vline(xintercept=xvar,col="green",alpha=0.5)+
scale_x_continuous(breaks=round(xvar,2))
    
print(gg)
dev.off()
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



## Choosing cut points and base reference

In [582]:
MELDNacut<-c(10,17,30)
MELDNab<-"[2.05,10]"

PRBCintraopcut <-c(3,5)
PRBCintraopb<-"[0,3]"


#Plateletsintraopcut <- 0.1
#Plateletsintraopb <- "[0,0.1]"


Durationcut <- 360
Durationb <- "[180,360]"

#Lactatescut <- c(3)
#Lactatesb <- "[0.5,3]"






cont.to.cat<-data.frame(id=c(1:nrow(df.cont)))

for (var in names(df.cont)[-1]) {
cat<-cut(df.cont[,var],
breaks=c(min(df.cont[,var]),
get(paste(var,"cut",sep="")),
max(df.cont[,var])),include.lowest= TRUE)
cat<-relevel(cat,ref=get(paste(var,"b",sep="")))
cont.to.cat<-cbind(cont.to.cat,cat)
}

## Calculating scores for each level

In [583]:
df.cont.to.cat<-cont.to.cat[,-1] 

names(df.cont.to.cat)<-names(df.cont)[-1]

df.final<-cbind(cbind(df.cont.to.cat,df.cat),

deg_cat_bin=df.cont$deg_cat_bin)

mod<-glm(deg_cat_bin~.,
df.final, family="binomial")
score<-round(coef(mod)[-1] * 10)


score.cont<-score[1:5]
score.cat<-score[6:length(score)]
 

In [584]:
summary(mod)


Call:
glm(formula = deg_cat_bin ~ ., family = "binomial", data = df.final)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4050  -0.8574   0.4203   0.7935   1.9066  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -0.3005     0.3448  -0.872 0.383352    
MELDNa(10,17]            0.7222     0.1818   3.972 7.13e-05 ***
MELDNa(17,30]            1.4095     0.2153   6.547 5.88e-11 ***
MELDNa(30,48.3]          2.2586     0.4195   5.383 7.31e-08 ***
PRBCintraop(3,5]         0.5621     0.2060   2.728 0.006364 ** 
PRBCintraop(5,45.3]      1.1368     0.2078   5.470 4.49e-08 ***
Duration(360,1.44e+03]  -0.1378     0.1713  -0.804 0.421301    
df.catIT126             -1.0952     0.3286  -3.333 0.000859 ***
df.catIT236              1.3685     1.1236   1.218 0.223214    
df.catIT271             -0.6824     0.5640  -1.210 0.226304    
df.catIT456             -1.2021     0.3473  -3.462 0.000537 ***
df.catIT630              0.

In [587]:
score

MELDNa(10,17]          MELDNa(17,30]        MELDNa(30,48.3] 
                     7                     14                     23 
      PRBCintraop(3,5]    PRBCintraop(5,45.3] Duration(360,1.44e+03] 
                     6                     11                     -1 
           df.catIT126            df.catIT236            df.catIT271 
                   -11                     14                     -7 
           df.catIT456            df.catIT630            df.catIT681 
                   -12                      7                      7

In [588]:
head(df.final)

MELDNa,PRBCintraop,Duration,df.cat,deg_cat_bin
"(10,17]","[0,3]","(360,1.44e+03]",IT126,1
"(10,17]","[0,3]","(360,1.44e+03]",IT126,1
"(10,17]","[0,3]","[180,360]",IT126,0
"(10,17]","[0,3]","[180,360]",IT126,0
"(17,30]","(5,45.3]","(360,1.44e+03]",IT126,1
"[2.05,10]","[0,3]","[180,360]",IT126,0


## Calculating scores for numeric variables

In [589]:
var.cont<-as.character(1)

for(var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.cont<-c(var.cont,var.red)
}

var.cont<-unique(var.cont)[-1]

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in names(score.cont)){
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
var.low<-as.numeric(str_extract(var,'(?<=(\\(|\\[))[0-9]+\\.*[0-9]*(?=\\,)'))
var.upper<-as.numeric(str_extract(var,'(?<=\\,)[0-9]+\\.*[0-9]*(?=\\])'))
df[,paste(var.red,"points",sep=".")]<-ifelse(
df[,var.red]<=var.upper&df[,var.red]>=var.low,
score[var],df[,paste(var.red,"points",sep=".")])
}

for(var in var.cont){
df[,paste(var,"points",sep=".")]<-ifelse(
is.na(df[,paste(var,"points",sep=".")]),
0,df[,paste(var,"points",sep=".")])
}

# Calculate scores for factor variables and take a sum

In [590]:
var.cat<-names(df.cat)

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in var.cat){
score.var<-score.cat[grep(var,names(score.cat))]
        names(score.var)<-sub(var,"",names(score.var))
    for(i in 1:(length(levels(df[,var]))-1)){
        df[,paste(var,"points",
        sep=".")]<-ifelse(
        df[,var]==names(score.var)[i]&
        is.na(df[,paste(var,"points",sep=".")]),
        score.var[i],
    df[,paste(var,"points",sep=".")])}
    }

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-ifelse(
    is.na(df[,paste(var,"points",sep=".")]),
    0,df[,paste(var,"points",sep=".")])}



In [591]:
df <- subset(df, select = -Centre)

In [592]:
df$score<-rowSums(df[,grepl("\\.+points",names(df))])

In [593]:
head(df)

,MELDNa,PRBCintraop,Duration,deg_cat_bin,dataset,MELDNa.points,PRBCintraop.points,score
NA.7,16.25053,2.531257,375.170,1,train,7,0,7
NA.160,14.77530,2.453829,373.250,1,train,7,0,7
NA.209,11.45819,2.150257,337.825,0,train,7,0,7
NA.493,12.89730,2.545714,358.495,0,validate,7,0,7
NA.519,11.50290,2.058971,345.235,0,train,7,0,7
NA.535,11.50650,2.292286,350.190,0,validate,7,0,7


## Converting the score to probability of mortality and comparing it to observed number of deaths

In [594]:
glmod<-glm(deg_cat_bin~score,
df[df$dataset=="train",],
family="binomial")

newx<-seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score))

prd<-predict(glmod,
newdata=data.frame(score=newx),
type="response",
se.fit=T)

count<-as.matrix(table(cut(df[df$dataset=="train",]$score,
breaks=seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score)),
include.lowest = T),
df[df$dataset=="train",]$deg_cat_bin))


In [595]:
summary(glmod)


Call:
glm(formula = deg_cat_bin ~ score, family = "binomial", data = df[df$dataset == 
    "train", ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3259  -1.1390   0.5637   0.9024   1.4743  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.67541    0.10985  -6.149 7.82e-10 ***
score        0.09739    0.00764  12.747  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1547.5  on 1181  degrees of freedom
Residual deviance: 1345.9  on 1180  degrees of freedom
AIC: 1349.9

Number of Fisher Scoring iterations: 4


## Visualization of the relationship between scores and probability of outcome events

In [596]:
par(mar=c(5,4,4,5)+.1)

barplot(t(count),
main="Scores versus probability on length of hospital stay",
xlab="Scores",
ylab="Observed number of patients",
space=0,
col=c("yellow","lightblue"))

legend("topleft",fill=c("yellow","lightblue",NA),
lty = c(NA,NA,1),lwd=c(NA,NA,2),
legend=c("fast","NotFast",
"Predicted Prob."),
col=c("black"),
border = c("black","black",NA))

par(new=TRUE)

plot(prd$fit~newx,
type="l",col="black",
lwd=2,xaxt="n",yaxt="n",
xlab="",ylab="")

polygon(c(rev(newx), newx),
c(rev(prd$fit+1.96*prd$se.fit),
prd$fit-1.96*prd$se.fit),
col = adjustcolor('grey80',alpha=0.5),
border = NA)

lines(newx, prd$fit+1.96*prd$se.fit,
lty = 'dashed', col = 'red')

lines(newx, prd$fit-1.96*prd$se.fit,
lty = 'dashed', col = 'red')

axis(4)

mtext("Predicted probability length of hospital stay",
side=4,line=3)


## Validation of the score

In [597]:
ddist <- datadist(df)

options(datadist='ddist')

f.score<-lrm(deg_cat_bin~score,
df[df$dataset=="train",],
x=TRUE,y=TRUE)

phat.score<-predict(f.score,
df[df$dataset=="validate",],
type="fitted")

v.score<-val.prob(phat.score,
df[df$dataset=="validate",]$deg_cat_bin,
m=20)


Warning message in min(xx[xx > upper]):
"no non-missing arguments to min; returning Inf"


# Prediction

In [621]:
df1 <- df[df$dataset == "validate",]

In [622]:
table(df1$deg_cat_bin)


  0   1 
140 259 

In [623]:
score

MELDNa(10,17]          MELDNa(17,30]        MELDNa(30,48.3] 
                     7                     14                     23 
      PRBCintraop(3,5]    PRBCintraop(5,45.3] Duration(360,1.44e+03] 
                     6                     11                     -1 
           df.catIT126            df.catIT236            df.catIT271 
                   -11                     14                     -7 
           df.catIT456            df.catIT630            df.catIT681 
                   -12                      7                      7

In [624]:
df1$predicted <- ifelse(df1$score > 9, 1,0)

ref <- as.factor(df1$deg_cat_bin)
pred <- as.factor(df1$predicted)

In [625]:
result <- confusionMatrix(pred,  ref) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0  82  66
         1  58 193
                                          
               Accuracy : 0.6892          
                 95% CI : (0.6413, 0.7343)
    No Information Rate : 0.6491          
    P-Value [Acc > NIR] : 0.05099         
                                          
                  Kappa : 0.3266          
                                          
 Mcnemar's Test P-Value : 0.52960         
                                          
            Sensitivity : 0.5857          
            Specificity : 0.7452          
         Pos Pred Value : 0.5541          
         Neg Pred Value : 0.7689          
             Prevalence : 0.3509          
         Detection Rate : 0.2055          
   Detection Prevalence : 0.3709          
      Balanced Accuracy : 0.6654          
                                          
       'Positive' Class : 0               
                              

Sensitivity          Specificity       Pos Pred Value 
           0.5857143            0.7451737            0.5540541 
      Neg Pred Value            Precision               Recall 
           0.7689243            0.5540541            0.5857143 
                  F1           Prevalence       Detection Rate 
           0.5694444            0.3508772            0.2055138 
Detection Prevalence    Balanced Accuracy 
           0.3709273            0.6654440